# TRABAJO FINAL PROCESAMIENTO DEL LENGUAJE NATURAL 
ALUMNOS : 

- ALEJANDRO MADRID GALARZA
- ANTONIO JOSÉ LÓPEZ MARTÍNEZ


### MOTIVACIÓN DEL TRABAJO
Vamos a realizar una aplicacion 'python' en 'jupyter-lab' para la asignatura de Procesamiento del Lenguaje Natural en la que trataremos, con los conocimientos adquiridos en la asignatura así como todo lo que sea necesario para la resolución del mismo, una aplicación que clasifique un conjunto de más de 3500 'tweets' en un clasificador de 11 clases que corresponden a 11 emociones distintas sobre las que clasificaremos.

In [192]:
# IMPORTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import spacy
import unidecode
#
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
#
from spacy.lang.es.stop_words import STOP_WORDS
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
#
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


2024-04-29 13:01:12.475942: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [147]:
# CARGAMOS LOS DATOS
train_data = pd.read_csv('Data/Train/sem_eval_train_es.csv')
test_data = pd.read_csv('Data/Test/sem_eval_test_grupo_10.csv')

In [148]:
train_data.shape, test_data.shape

((3561, 13), (679, 2))

In [149]:
# Vamos a eliminar los id's de ambos csv
train_data = train_data.drop(['ID'], axis=1)
test_data = test_data.drop(['ID'], axis=1)
print(train_data.shape, test_data.shape)
print(train_data.head())
print(test_data.head())

(3561, 12) (679, 1)
                                               Tweet  anger  anticipation  \
0  @aliciaenp Ajajjaa somos del clan twitteras pe...  False         False   
1  @AwadaNai la mala suerte del gato fichame la c...  False         False   
2  @audiomano A mí tampoco me agrado mucho eso. E...   True         False   
3  Para llevar a los bebes de un lugar a otro deb...  False         False   
4  @DalasReview me encanta la terrible hipocresia...   True         False   

   disgust   fear    joy   love  optimism  pessimism  sadness  surprise  trust  
0    False  False   True  False     False      False    False     False  False  
1    False   True  False  False     False       True    False     False  False  
2    False  False  False  False     False      False    False     False  False  
3    False  False   True  False     False      False    False     False  False  
4     True  False  False  False     False      False    False     False  False  
                               

Ahora que hemos eliminado la columna de ID's de los conjuntos de entrenamiento y testeo tenemos que separar nuestros tweets de testeo que, como podemos observar tiene 12 dimensiones (11 de las diferentes emociones + 1 para los tweets). 
Los vamos a separar como: 
- $X=contenido$ $tweets$
- $Y=categoría$ $tweet$

In [150]:
X = train_data['Tweet']
y = train_data.drop(['Tweet'], axis=1)
print(X.shape, y.shape, '\n')
print(X.head(), '\n')
print(y.head())

(3561,) (3561, 11) 

0    @aliciaenp Ajajjaa somos del clan twitteras pe...
1    @AwadaNai la mala suerte del gato fichame la c...
2    @audiomano A mí tampoco me agrado mucho eso. E...
3    Para llevar a los bebes de un lugar a otro deb...
4    @DalasReview me encanta la terrible hipocresia...
Name: Tweet, dtype: object 

   anger  anticipation  disgust   fear    joy   love  optimism  pessimism  \
0  False         False    False  False   True  False     False      False   
1  False         False    False   True  False  False     False       True   
2   True         False    False  False  False  False     False      False   
3  False         False    False  False   True  False     False      False   
4   True         False     True  False  False  False     False      False   

   sadness  surprise  trust  
0    False     False  False  
1    False     False  False  
2    False     False  False  
3    False     False  False  
4    False     False  False  


### Preprocesado de datos
Ahora solo vamos a trabajar con el conjunto de entrenamiento ya que tenemos que masticar los datos y pasárselos al modelo que posteriormente entrenamremos

In [151]:
# Comenzamos el preprocesado cargando el modelo a ejecutar
nlp=spacy.load('Data/Model/es_core_news_sm-3.7.0-py3-none-any/es_core_news_sm/es_core_news_sm-3.7.0')

In [152]:
# En esta celda vamos a definir las funciones para la limpieza del texto
nltk.download('punkt')
nltk.download('stopwords')
def limpiar_texto(tweet):
    # Elimina los acentos y convierte caracteres especiales
    tweet = unidecode.unidecode(tweet)
    # Elimina menciones y hashtags
    tweet = re.sub(r'@[A-Za-z0-9_]+', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    # Tokeniza el tweet
    tokens = word_tokenize(tweet, language='spanish')
    # Elimina stop words y palabras de 1 letra, y realiza stemming
    stop_words = set(stopwords.words('spanish'))
    stemmer = SnowballStemmer('spanish')
    tokens = [stemmer.stem(word) for word in tokens if word.lower() not in stop_words and len(word) > 1]
    return " ".join(tokens)

[nltk_data] Downloading package punkt to /Users/alejandro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alejandro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [158]:
# Limpiamos nuestros tweets
tweets_limpios = []
for tweet in X: 
    tweet_limpio = limpiar_texto(tweet)
    tweets_limpios.append(tweet_limpio)

In [182]:
X = np.array(tweets_limpios)
X

array(['ajajj clan twitt perd pa event `` import ``',
       'mal suert gat ficham car help pls',
       'tampoc agrad especial trat justif', ...,
       'prim anos enfad conmig explic azul sol chic ros chic ensen',
       'jajaj ... see', 'qui abraz qui quer hac falt carin gent'],
      dtype='<U106')

In [183]:
# Ahora que tenemos los datos limpios vamos a separar en entrenamiento y testeo para el modelo 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [184]:
# Creamos la representación vecrotial de nuestros datos por el método TF-IDF
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
# Además vamos a transformar nuestros y en 1-D para los modelos
y_train = y_train.idxmax(axis=1)
y_test = y_test.idxmax(axis=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2848, 4754) (713, 4754) (2848,) (713,)


Una vez creada la BOW, en nuestro caso por el método TF-IDF tenemos que crear el modelo que usaremos para la predicció y entrenarlo con nuestros datos de entrenamiento. 
### Modelo
En nuestro caso vamos a realizar un pequeño estudio sobre 3 modelos para saber cuál es mejor para nuestro Dataset y luego decidiremos el mejor y lo usaremospara la predicción. 
Los modelos a tratar va a ser:
- Multinomial Naive Bayes
- SVM
- Transformer --> BERT

#### Naive Bayes

In [188]:
# Creamos el modelo Naive Bayes
nbModelo = MultinomialNB()
ajusteNB = nbModelo.fit(X_train, y_train)           # Entrenamos el modelo
nBScore = nbModelo.score(X_test, y_test)            # Sacamos la precisión del modelo
nBScore

0.46563814866760167

Como esta precisión es bastante mala vamos a probar con la validación cruzada

In [189]:
# Repetimos el proceso pero con validación cruzada
nBModelo = MultinomialNB()
scores = cross_val_score(nbModelo, X_train, y_train, cv=5)
print(f"Puntuaciones: {scores}\n")
print(f'Puntuación media: {scores.mean()}\n')
print(f'Desviación Estandar: {scores.std()}\n')


Puntuaciones: [0.45964912 0.48070175 0.45964912 0.48857645 0.48330404]

Puntuación media: 0.4743760984182777

Desviación Estandar: 0.012289342594939669



Con la validación cruzada el número aumenta pero no es nada sustancial como para que podemos usar este modelo para nuesto dataset

#### Máquina Vector Soporte (SVC)

In [191]:
# Creamos el modelo y lo aplicamos a nuestros datos
modeloSVC = SVC(kernel='linear')
modeloSVC.fit(X_train, y_train)
y_predicciones = modeloSVC.predict(X_test) # Predecimos el conjunto de testeo del entrenamiento
print(f'SVM: Classification Report: {classification_report(y_test, y_predicciones)}')



SVM: Classification Report:               precision    recall  f1-score   support

       anger       0.52      0.86      0.65       263
anticipation       0.45      0.11      0.18        79
     disgust       0.00      0.00      0.00        27
        fear       0.80      0.49      0.61        49
         joy       0.57      0.64      0.60       166
        love       0.00      0.00      0.00        15
    optimism       0.00      0.00      0.00        16
   pessimism       0.38      0.07      0.11        45
     sadness       0.65      0.48      0.55        42
    surprise       0.00      0.00      0.00         9
       trust       0.00      0.00      0.00         2

    accuracy                           0.55       713
   macro avg       0.31      0.24      0.25       713
weighted avg       0.49      0.55      0.48       713



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

Como podemos observar, estamos teniendo problemas con este modelo ya que tenemos precisiones de 0% en algunas de las categorías así como que la media de accuracy de nuestro modelo es de 53%

### Redes Recurrentes

Como estamos teniendo muchos problemas con los modelos previos, vamos a pasarnos al ámbito de las redes neuronales, concretamente en las redes neuronales recurrentes.

##### Long-Short Term Memory (LSTM)
Esta va a ser la primera red recurrente que vamos a probar ya que es eficaz a la hora de capturar las dependencias a largo plao en datos secuenciales.

Para este modelo necestamos sacar los datos de tipo texto otra vez, ya que este modelo se encarga de hacer todo

In [194]:
X = train_data['Tweet']
y = train_data.drop(['Tweet'], axis=1)
print(X.shape, y.shape, '\n')
print(X.head(), '\n')
print(y.head())

(3561,) (3561, 11) 

0    @aliciaenp Ajajjaa somos del clan twitteras pe...
1    @AwadaNai la mala suerte del gato fichame la c...
2    @audiomano A mí tampoco me agrado mucho eso. E...
3    Para llevar a los bebes de un lugar a otro deb...
4    @DalasReview me encanta la terrible hipocresia...
Name: Tweet, dtype: object 

   anger  anticipation  disgust   fear    joy   love  optimism  pessimism  \
0  False         False    False  False   True  False     False      False   
1  False         False    False   True  False  False     False       True   
2   True         False    False  False  False  False     False      False   
3  False         False    False  False   True  False     False      False   
4   True         False     True  False  False  False     False      False   

   sadness  surprise  trust  
0    False     False  False  
1    False     False  False  
2    False     False  False  
3    False     False  False  
4    False     False  False  


In [197]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(2848,) (2848, 11) (713,) (713, 11)


In [204]:
# Vamos a tokenizar
max_words = 1000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_seq, maxlen=1000)
X_train_padded.shape

(2848, 1000)

In [205]:
# Definimos el modelo LSTM
model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=max_words, output_dim=100, input_length=100))
model_lstm.add(LSTM(units=64))
model_lstm.add(Dense(units=11, activation='softmax'))  # Ajusta num_classes según tu problema
model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [206]:
model_lstm.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 23s 288ms/step - accuracy: 0.2490 - loss: 3.9506 - val_accuracy: 0.2263 - val_loss: 4.0119
Epoch 2/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 21s 287ms/step - accuracy: 0.2998 - loss: 4.0010 - val_accuracy: 0.2263 - val_loss: 3.9880
Epoch 3/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 21s 296ms/step - accuracy: 0.3049 - loss: 3.8937 - val_accuracy: 0.4105 - val_loss: 3.9442
Epoch 4/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 27s 381ms/step - accuracy: 0.3366 - loss: 3.8697 - val_accuracy: 0.4105 - val_loss: 3.8956
Epoch 5/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 24s 328ms/step - accuracy: 0.3450 - loss: 3.7979 - val_accuracy: 0.4105 - val_loss: 3.9095
Epoch 6/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 23s 317ms/step - accuracy: 0.2905 - loss: 3.7671 - val_accuracy: 0.2263 - val_loss: 3.9025
Epoch 7/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 23s 318ms/step - accuracy: 0.2820 - loss: 3.6842 - val_accuracy: 0.4105 - val_loss: 3.9119
Epoch 8/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 27s 380ms/step - accuracy: 0.2964 - loss: 3.8030 - val_accu

KeyboardInterrupt: 

#### Red Neuronal Recurrente Convolucional (CRNN) 
Esta va a ser la segunda red neuronal que vamos a probar. Estas aplican ambos tipos de redes neuronales para capturar información tanto secuencial como local.